In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from hyperopt import fmin, tpe, hp

df_train = pd.read_csv('./data/train.csv')

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

## Features 

In [22]:
# Agrupaciones

df_train_f = df_train.copy()

# Zona Provincias
provincias_sur = ['Yucatán','Quintana Roo','Campeche','Tabasco', 'Veracruz', 'Oaxaca', 'Chiapas', 'Guerrero', 'Michoacán']
provincias_centro = ['Tlaxcala','Morelos','Puebla','Hidalgo','Aguascalientes', 'Distrito Federal', 'Jalisco', 'Edo. de México','Querétaro', 'Colima', 'Nayarit', 'Aguascalientes', 'Guanajuato', 'Hidalgo']
provincias_norte = ['Sonora','San luis Potosí','Baja California Norte', 'Baja California Sur', 'Chihuahua', 'Durango', 'Coahuila', 'Sinaloa', 'Nuevo León',  'Tamaulipas']


prov_por_precio = {'Distrito Federal':0, 'Edo. de México':1, 'Nuevo León':2, 'Guerrero':3, 'Quintana Roo':4, 'Jalisco':5, 'Tabasco':6, 'Puebla':6, 'Morelos':6, 'Querétaro':7, 'Yucatán':8, 'Chiapas':9, 'San luis Potosí':9, 'Veracruz':10, 'Guanajuato':11, 'Chihuahua':12, 'Sinaloa':13, 'Tamaulipas':14, 'Oaxaca':14, 'Campeche':14, 'Colima':15, 'Coahuila':15, 'Sonora':16, 'Michoacán':16, 'Aguascalientes':16, 'Zacatecas':17, 'Nayarit':18, 'Hidalgo':19, 'Baja California Sur':20, 'Durango':20}
def top_provincias(x):
    if x in prov_por_precio:
        return prov_por_precio[x]
    else:
        return np.nan


# Intervsalos metros
punto20 = df_train_f['metrostotales'].quantile(0.2)
punto40 = df_train_f['metrostotales'].quantile(0.4)
punto60 = df_train_f['metrostotales'].quantile(0.6)
punto80 = df_train_f['metrostotales'].quantile(0.8)
punto20_cubiertos = df_train_f['metroscubiertos'].quantile(0.2)
punto40_cubiertos = df_train_f['metroscubiertos'].quantile(0.4)
punto60_cubiertos = df_train_f['metroscubiertos'].quantile(0.6)
punto80_cubiertos = df_train_f['metroscubiertos'].quantile(0.8)

# Division de tipo de propiedad
propiedades_compartidas = ['Casa en condominio', 'Duplex', 'Departamento Compartido','Lote']
propiedades_frecuentes = ['Casa', 'Apartamento', 'Edificio']

# Escuelas y Centros Comerciales Cercanos
def cercanos(x):
    if not x['escuelascercanas'] and not x['centroscomercialescercanos']:
        return 'Ninguno'
    if x['escuelascercanas'] and not x['centroscomercialescercanos']:
        return 'Centro Comerciales'
    if not x['centroscomercialescercanos'] and x['escuelascercanas']:
        return 'Escuelas Cercanas'
    return 'Ambos'

# Delincuencia
ciudades_mas_peligrosas = pd.read_csv('./data/ciudades_mas_peligrosas.csv')
df_crimenes = pd.merge(df_train_f, ciudades_mas_peligrosas, on = 'ciudad', how= 'left')

# Turismo
ciudades_mas_turisticas = pd.read_csv('./data/ciudades_mas_turisticas.csv')
df_turismo = pd.merge(df_train_f, ciudades_mas_turisticas, on = 'ciudad', how= 'left')

# Personas por propiedad
def calculo_inquilinos(x):
    habitaciones = x['habitaciones']
    banos = x['banos']
    if habitaciones == 0 | banos == 0 | habitaciones == np.nan | banos == np.nan:
        return 0
    if habitaciones == 1 & banos == 1:
        return 1
    if habitaciones == 2 & banos == 1:
        return 3
    if habitaciones == 2 & banos == 2:
        return 4
    if habitaciones == 3 & banos == 1:
        return 5
    if habitaciones == 3 & banos == 2:
        return 6
    if habitaciones == 4 & banos == 1:
        return 6
    if habitaciones == 4 & banos == 2:
        return 7
    if habitaciones == 4 & (banos == 3 | banos == 4):
        return 8
    if habitaciones == 5:
        return 10
    if habitaciones > 5:
        return 11

# Ciudad Centrica por provincia
ciudades_centricas = pd.read_csv('./data/ciudadcentrica_por_provincia.csv')
df_centro = pd.merge(df_train_f, ciudades_centricas, on = 'ciudad', how= 'left')

In [23]:
# Por Tipo de Propiedad
df_train_f['escomercial'] = df_train_f.tipodepropiedad.str.contains('.*omercial.*').astype(bool)
df_train_f['promedio_metros_tipo_propiedad'] = df_train_f.groupby(['tipodepropiedad'])['metrostotales'].transform('mean')
df_train_f['promedio_metros_tipo_propiedad'] = df_train_f.groupby(['tipodepropiedad'])['metroscubiertos'].transform('mean')
df_train_f['promedio_precio_tipo_propiedad'] = df_train_f.groupby(['tipodepropiedad'])['precio'].transform('mean')
df_train_f['tipo_propiedad_compartida'] = list(map(lambda x: True if x in propiedades_compartidas else False, df_train_f['tipodepropiedad']))
df_train_f['prop_frecuente'] = list(map(lambda x: True if x in propiedades_frecuentes else False, df_train_f['tipodepropiedad'])) 

In [24]:
# Por metros cubiertos y totales
df_train_f['porcentaje_metros'] = df_train_f['metroscubiertos']/df_train_f['metrostotales']
df_train_f['diferencia_metros'] = df_train_f['metrostotales'] - df_train_f['metroscubiertos']
df_train_f['intervalo_metros_totales'] = list(map(lambda x: 5 if x<punto20 else (4 if punto20<x<punto40 else (3 if punto40<x<punto60 else(2 if punto60<x<punto80 else 1))), df_train_f['metrostotales']))
df_train_f['intervalo_metros_cubiertos'] = list(map(lambda x: 5 if x<punto20_cubiertos else (4 if punto20_cubiertos<x<punto40_cubiertos else (3 if punto40_cubiertos<x<punto60_cubiertos else(2 if punto60_cubiertos<x<punto80_cubiertos else 1))), df_train_f['metrostotales']))

In [25]:
# Por ubicación
df_train_f['promedio_precio_ciudad'] = df_train_f.groupby(['provincia', 'ciudad'])['precio'].transform('mean')
df_train_f['promedio_id_zona'] = df_train_f.groupby(['idzona'])['precio'].transform('mean')
df_train_f['varianza_precio_ciudad'] = df_train_f.groupby(['provincia', 'ciudad'])['precio'].transform('std')
df_train_f['varianza_id_zona'] = df_train_f.groupby(['idzona'])['precio'].transform('std')
df_train_f['zona'] = list(map(lambda x: 'Norte ' if x in provincias_norte else ('Sur' if x in provincias_sur else 'Centro'), df_train_f['provincia']))
df_train_f['top_provincia'] = df_train_f['provincia'].apply(top_provincias)
df_train_f['es_ciudad_centrica'] = df_centro['centro'].replace({np.nan:False})
df_train_f['promedio_metros_provincia'] = df_train_f.groupby(['provincia'])['metrostotales'].transform('mean')
df_train_f['promedio_metros_provincia'] = df_train_f.groupby(['provincia'])['metroscubiertos'].transform('mean')

In [26]:
# Por fechas
df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha'])
df_train_f['anio'] = df_train_f['fecha'].dt.year
df_train_f['mes'] = df_train_f['fecha'].dt.month
df_train_f['dia'] = df_train_f['fecha'].dt.day
df_train_f['promedio_por_mes'] = df_train_f.groupby(['anio','mes'])['precio'].transform('mean')
df_train_f['varianza_por_mes'] = df_train_f.groupby(['anio','mes'])['precio'].transform('std')
df_train_f['trimestre'] = (df_train_f['fecha'].dt.month)%3+1


In [27]:
# Por habitaciones, garages y banos
df_train_f['promedio_precio_habitaciones'] = df_train_f.groupby(['habitaciones'])['precio'].transform('mean')
df_train_f['promedio_precio_habitaciones_banos_garages'] = df_train_f.groupby(['habitaciones', 'banos', 'garages'])['precio'].transform('mean')
df_train_f['promedio_precio_banos_garages'] = df_train_f.groupby(['banos', 'garages'])['precio'].transform('mean')
#df_train_f['cantidad_inquilinos'] = df_train_f[['habitaciones', 'banos']].apply(calculo_inquilinos,axis=1)

In [28]:
# Booleanos
df_train_f['promedio_precio_booleanos'] = df_train_f.groupby(['gimnasio', 'usosmultiples', 'piscina'])['precio'].transform('mean')
df_train_f['escualas_centros_cercanos'] = df_train_f[['centroscomercialescercanos', 'escuelascercanas']].apply(cercanos, axis=1)
df_train_f['delincuencia'] = df_crimenes['peligro'].replace({np.nan:False})
df_train_f['turismo'] = df_turismo['turismo'].replace({np.nan:False})
df_train_f['es_antigua'] = list(map(lambda x: True if x>30 else False, df_train_f['antiguedad']))


In [29]:
# TITULOS/DESCRIPCIONES/DISTANCIA A COMERCIOS

df_train_f

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,varianza_por_mes,trimestre,promedio_precio_habitaciones,promedio_precio_habitaciones_banos_garages,promedio_precio_banos_garages,promedio_precio_booleanos,escualas_centros_cercanos,delincuencia,turismo,es_antigua
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,2.195410e+06,3,1.616900e+06,1.943985e+06,1.742841e+06,2.398349e+06,Ninguno,False,False,False
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,1.813118e+06,1,2.769607e+06,2.196152e+06,2.399207e+06,2.398349e+06,Ambos,False,False,False
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,2.200963e+06,2,2.769607e+06,2.196152e+06,2.399207e+06,2.398349e+06,Ninguno,False,False,False
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,1.842065e+06,1,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ambos,False,False,False
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,2.252515e+06,1,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ninguno,False,False,False
5,126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,...,1.892367e+06,1,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ambos,False,False,False
6,139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,...,2.285987e+06,3,2.769607e+06,1.612232e+06,1.742841e+06,2.398349e+06,Ninguno,False,False,False
7,5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,...,2.315079e+06,2,3.494389e+06,3.055612e+06,3.261308e+06,2.398349e+06,Ninguno,False,True,False
8,44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,...,1.882606e+06,2,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ambos,False,False,False
9,134537,terreno,"terreno de 5.500m2 bardeado, uso de suelo h-20...",Terreno,Av. Morelos,Ixtapaluca,Edo. de México,NaN,NaN,NaN,...,2.354765e+06,1,NaN,NaN,NaN,2.398349e+06,Ninguno,False,False,False


In [30]:
# Insight
zona = df_train
punto2 = zona['precio'].quantile(0.2)
punto4 = zona['precio'].quantile(0.4)
punto6 = zona['precio'].quantile(0.6)
punto8 = zona['precio'].quantile(0.8)

puntajes = zona.dropna(subset = ['metrostotales', 'metroscubiertos']) #poner promedio

In [31]:
zona_ciudad_casa = puntajes.groupby('ciudad').agg('mean')

zona_ciudad_casa['grupoCiudad'] = list(map(lambda x: 'Grupo 1' if x < punto2 else ('Grupo 2' if punto2<x<punto4 else ('Grupo 3' if punto4<x<punto6 else ('Grupo 4' if punto6<x<punto8 else 'Grupo 5'))), zona_ciudad_casa["precio"]))
zona_ciudad_casa_un= zona_ciudad_casa.unstack()

In [32]:
zona_metros_casa = puntajes.groupby('metroscubiertos').agg('mean')
zona_metros_casa['grupoMetros'] = list(map(lambda x: 'Grupo 1' if x < punto2 else ('Grupo 2' if punto2<x<punto4 else ('Grupo 3' if punto4<x<punto6 else ('Grupo 4' if punto6<x<punto8 else 'Grupo 5'))), zona_metros_casa["precio"]))
zona_metros_casa_un = zona_metros_casa.unstack()

In [33]:
zona_metros_cub_casa = puntajes.groupby('metrostotales').agg('mean')
zona_metros_cub_casa['grupoCubiertos'] = list(map(lambda x: 'Grupo 1' if x < punto2 else ('Grupo 2' if punto2<x<punto4 else ('Grupo 3' if punto4<x<punto6 else ('Grupo 4' if punto6<x<punto8 else 'Grupo 5'))), zona_metros_cub_casa["precio"]))
zona_metros_cub_casa_un = zona_metros_cub_casa.unstack()

In [34]:
provincias1 = ['Guerrero', 'Tabasco', 'Nueva León', 'Quintana Roo', 'Morelos', 'Jalisco', 'Puebla']
provincias2 = ['Querétaro', 'Yucatán', 'Chiapas', 'Guanajuato', 'San Luis Potosí', 'Veracruz', 'Chihuahua', 'Sinaloa']
provincias3 = ['Tamaulipas', 'Oaxaca', 'Campeche', 'Campeche', 'Zacatecas', 'Colima', 'Aguascalientes', 'Michoacán', 'Sonora', 'Nayarit'] 

def df_puntajes(zona):
    for i in range(len(zona)):
        puntaje = 0
        if (zona.piscina[i] == np.nan) | (zona.piscina[i] == np.nan) | (zona.usosmultiples[i] == np.nan)| (zona.gimnasio[i] == np.nan) |(zona.banos[i] == np.nan)|(zona.habitaciones[i] == np.nan) | (zona.ciudad[i] not in zona_ciudad_casa.index) | (zona.metroscubiertos[i] not in zona_metros_cub_casa.index)| (zona.metrostotales[i] not in zona_metros_casa.index) :
            zona['puntaje'][i] = puntaje
            continue
        if zona.piscina[i] == 1.0:
            puntaje += 8
        if zona.usosmultiples[i] == 1.0:
            puntaje +=5
        if zona.gimnasio[i] == 1.0:
            puntaje +=5
        if zona.banos[i] == 2.0:
            puntaje +=2
        elif zona.banos[i] == 3.0:
            puntaje += 5
        elif zona.banos[i] == 4.0:
            puntaje += 6
        if zona.habitaciones[i] == 2.0:
            puntaje += 2
        elif zona.habitaciones[i] == 3.0:
            puntaje += 5
        elif zona.habitaciones[i] > 3.0:
            puntaje += 6
        if zona_ciudad_casa_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 2':
            puntaje += 2
        elif zona_ciudad_casa_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 3':
            puntaje += 5
        elif zona_ciudad_casa_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 4':
            puntaje += 8
        elif zona_ciudad_casa_un[('grupoCiudad',zona.ciudad[i])] == 'Grupo 5':
            puntaje += 10
        if zona_metros_casa_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 2':
            puntaje += 4
        elif zona_metros_casa_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 3':
            puntaje += 6
        elif zona_metros_casa_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 4':
            puntaje += 8
        elif zona_metros_casa_un[('grupoMetros',zona.metrostotales[i])] == 'Grupo 5':
            puntaje += 10
        if zona_metros_cub_casa_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 2':
            puntaje += 4
        elif zona_metros_cub_casa_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 3':
            puntaje += 6
        elif zona_metros_cub_casa_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 4':
            puntaje += 8
        elif zona_metros_cub_casa_un[('grupoCubiertos',zona.metroscubiertos[i])] == 'Grupo 5':
            puntaje += 10
        if zona.provincia[i] == 'Distrito Federal':
            puntaje += 8
        elif zona.provincia[i] == 'Edo. de México':
            puntaje += 7
        elif zona.provincia[i] in provincias1:
            puntaje += 5
        elif zona.provincia[i] in provincias2:
            puntaje += 4
        elif zona.provincia[i] in provincias3:
            puntaje += 2
        zona['puntaje'][i] = puntaje

In [35]:
zona['puntaje']=0
imp = SimpleImputer()
zona['metroscubiertos'] = imp.fit_transform(zona[['metroscubiertos']])
zona['metrostotales'] = imp.fit_transform(zona[['metrostotales']])
zona = zona.reset_index()

In [36]:
#df_puntajes(zona)
df_train_f['puntaje'] = zona['puntaje']

df_train_f

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,trimestre,promedio_precio_habitaciones,promedio_precio_habitaciones_banos_garages,promedio_precio_banos_garages,promedio_precio_booleanos,escualas_centros_cercanos,delincuencia,turismo,es_antigua,puntaje
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,3,1.616900e+06,1.943985e+06,1.742841e+06,2.398349e+06,Ninguno,False,False,False,0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,1,2.769607e+06,2.196152e+06,2.399207e+06,2.398349e+06,Ambos,False,False,False,0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,2,2.769607e+06,2.196152e+06,2.399207e+06,2.398349e+06,Ninguno,False,False,False,0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,1,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ambos,False,False,False,0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,1,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ninguno,False,False,False,0
5,126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,...,1,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ambos,False,False,False,0
6,139233,de oportunidad casa en san lorenzo,"ubicada en esquina, pertenece san lorenzo agen...",Casa,NaN,Oaxaca de Juárez,Oaxaca,NaN,3.0,1.0,...,3,2.769607e+06,1.612232e+06,1.742841e+06,2.398349e+06,Ninguno,False,False,False,0
7,5013,casa emilia en venta en selvamar playa del carmen,casa emilia en venta playa del carmenfracciona...,Casa,condominio el trebol,Playa del Carmen,Quintana Roo,2.0,4.0,2.0,...,2,3.494389e+06,3.055612e+06,3.261308e+06,2.398349e+06,Ninguno,False,True,False,0
8,44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,...,2,1.616900e+06,8.013918e+05,9.232625e+05,2.398349e+06,Ambos,False,False,False,0
9,134537,terreno,"terreno de 5.500m2 bardeado, uso de suelo h-20...",Terreno,Av. Morelos,Ixtapaluca,Edo. de México,NaN,NaN,NaN,...,1,NaN,NaN,NaN,2.398349e+06,Ninguno,False,False,False,0


## LightBM

In [41]:
import lightgbm as lgb

In [ ]:
df_gbm = df_train_f[['precio', 'metroscubiertos', 'lat', 'lng', 'metrostotales', 'antiguedad', 'garages',
                     'banos', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 
                     'centroscomercialescercanos', 'escomercial', 'promedio_precio_ciudad', 'promedio_id_zona',
                     'promedio_metros_tipo_propiedad', 'promedio_precio_tipo_propiedad',
                     'tipo_propiedad_compartida', 'prop_frecuente', 'porcentaje_metros',
       'diferencia_metros', 'intervalo_metros_totales',
       'intervalo_metros_cubiertos', 'promedio_precio_ciudad',
       'promedio_id_zona', 'varianza_precio_ciudad', 'varianza_id_zona',
       'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_provincia', 'anio', 'mes', 'dia', 'promedio_por_mes',
       'varianza_por_mes', 'trimestre', 'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_booleanos',
       'delincuencia', 'turismo', 'es_antigua']]


df_gbm_test, df_gbm_train = train_test_split(df_gbm, test_size=0.25, random_state=1)

y_test = df_gbm_test['precio']
y_train = df_gbm_train['precio']
x_test = df_gbm_test.drop('precio', axis=1)
x_train = df_gbm_train.drop('precio', axis=1)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
#     'metric': {'l2', 'l1'},
    'metric': '', # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=1000,
                early_stopping_rounds=5)

y_pred_test = gbm.predict(x_test, num_iteration=gbm.best_iteration)
y_pred_train = gbm.predict(x_train, num_iteration=gbm.best_iteration)

gbm_rmsle_train = RMSLE(y_train, y_pred_train)
gbm_rmsle = RMSLE(y_test, y_pred_test)

print(f"RMSLE LightGBM: {gbm_rmsle_train:.5f}")
print(f"RMSLE LightGBM: {gbm_rmsle:.5f}")

[1]	valid_0's l2: 4.35029e+12
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 4.07354e+12
[3]	valid_0's l2: 3.82153e+12
[4]	valid_0's l2: 3.59254e+12
[5]	valid_0's l2: 3.39351e+12
[6]	valid_0's l2: 3.20353e+12
[7]	valid_0's l2: 3.03046e+12
[8]	valid_0's l2: 2.87114e+12
[9]	valid_0's l2: 2.72721e+12
[10]	valid_0's l2: 2.59607e+12
[11]	valid_0's l2: 2.47535e+12
[12]	valid_0's l2: 2.36486e+12
[13]	valid_0's l2: 2.26327e+12
[14]	valid_0's l2: 2.17009e+12
[15]	valid_0's l2: 2.08544e+12
[16]	valid_0's l2: 2.00861e+12
[17]	valid_0's l2: 1.93852e+12
[18]	valid_0's l2: 1.8742e+12
[19]	valid_0's l2: 1.81585e+12
[20]	valid_0's l2: 1.75804e+12
[21]	valid_0's l2: 1.70351e+12
[22]	valid_0's l2: 1.65405e+12
[23]	valid_0's l2: 1.60944e+12
[24]	valid_0's l2: 1.56554e+12
[25]	valid_0's l2: 1.52797e+12
[26]	valid_0's l2: 1.4927e+12
[27]	valid_0's l2: 1.46069e+12
[28]	valid_0's l2: 1.43126e+12
[29]	valid_0's l2: 1.40241e+12
[30]	valid_0's l2: 1.37602e+12
[31]	valid_0's l2: 1.

In [ ]:
df_test_f = df_test.copy()

df_test_f['escomercial'] = df_test_f.tipodepropiedad.str.contains('.*omercial.*').astype(bool)

df_promedios = df_train_f.groupby(['provincia', 'ciudad']).agg({'precio':'mean'}).rename(columns = {'precio' : 'promedio_precio_ciudad'}).reset_index()
df_test_f = pd.merge(df_test, df_promedios, on=['provincia', 'ciudad'])

df_test_f = df_test_f.drop(['titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad', 'provincia', 'fecha'], axis=1)

y_pred_test = gbm.predict(df_test_f, num_iteration=gbm.best_iteration)

df_test_f['precio'] = y_pred_test

df_test_f[['id', 'precio']].to_csv('respuesta.csv', header=False, index=False)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [ ]:
df_test.head()

In [ ]:
print('Loading data...')
df_gbm = df_train_f[['precio', 'metroscubiertos', 'lat', 'lng', 'metrostotales', 'antiguedad', 'garages',
                     'banos', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 
                     'centroscomercialescercanos', 'escomercial', 'promedio_precio_ciudad']]

df_gbm_test, df_gbm_train = train_test_split(df_gbm, test_size=0.25, random_state=1)

y_test = df_gbm_test['precio']
y_train = df_gbm_train['precio']
X_test = df_gbm_test.drop('precio', axis=1)
X_train = df_gbm_train.drop('precio', axis=1)


In [ ]:
print('Starting training...')
# train
gbm = lgb.LGBMRegressor(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5)

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

# feature importances
print('Feature importances:', list(gbm.feature_importances_))


# self-defined eval metric
# f(y_true: array, y_pred: array) -> name: string, eval_result: float, is_higher_better: bool
# Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False


print('Starting training with custom eval function...')
# train
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=rmsle,
        early_stopping_rounds=5)


# another self-defined eval metric
# f(y_true: array, y_pred: array) -> name: string, eval_result: float, is_higher_better: bool
# Relative Absolute Error (RAE)
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False


print('Starting training with multiple custom eval functions...')
# train
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=lambda y_true, y_pred: [rmsle(y_true, y_pred), rae(y_true, y_pred)],
        early_stopping_rounds=5)

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
print('The rmsle of prediction is:', rmsle(y_test, y_pred)[1])
print('The rae of prediction is:', rae(y_test, y_pred)[1])

# other scikit-learn modules
estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)

print('Best parameters found by grid search are:', gbm.best_params_)